**2.92**

The key part is not just how to write code and implement every detail, it is about thinking in a same way that the chapter2 has showed which is :*** how to apply the ideas of abstract data and generic operations to help organize this effort ***.

We will introduce a new data structure of polynomial that can support and express multiple variables. We will follow the same way of data abstraction in the book.

In [ ]:
(define (add-poly-mult-var p1 p2)
  (make-poly-mult-var (union (var-set p1)  (var-set p2))
                      (add-terms (term-list p1) (term-list p2))))

(define (mul-poly-mult-var p1 p2)
  (make-poly-mult-var (union (var-set p1)  (var-set p2))
                      (mul-terms (term-list p1)(term-list p2))))


In [ ]:
(define (install-polynomial-mult-var-package)
  ;; internal procedures
  ;; representation of poly
  (define (make-poly var-set term-list)
    (list var-set term-list))
  (define (var-set p) (car p))
  (define (term-list p) (cdar p))

  ;; representation of terms and term lists
  <procedures adjoin-term ...coeff from text below>

  ;; continued on next page

  (define (add-poly-mult-var p1 p2) ...)

  (define (mul-poly-mult-var p1 p2) ...)

  ;; interface to rest of the system
  (define (tag p) (attach-tag 'poly-mult-var p))
  (put 'add '(poly-mult-var poly-mult-var) 
       (lambda (p1 p2) (tag (add-poly-mult-var p1 p2))))
  (put 'mul '(poly-mult-var poly-mult-var) 
       (lambda (p1 p2) (tag (mul-poly-mult-var p1 p2))))
  (put 'make 'poly-mult-var
       (lambda (var-list terms) (tag (make-poly var-list terms))))

  'done)
;; Outside package
(define (make-poly-mult-var V T)
  ((get 'make 'poly-mult-var) V T))

Like the textbook, we now implement `add-terms`. Original terms are constructed with `order` and `coeff`, now this term is constructed with `key` and `value` pairs in `Map`.

We assume that terms are `Map`, which has a predicate `has?`, a selector `get` and procedure `set`

In [1]:
(define (add-terms L1 L2)
  (cond ((empty-termlist? L1) L2)
        ((empty-termlist? L2) L1)
        (else
         (let ((t1 (first-term L1)))
           (if (has? L2 (key t1))
               (let ((new-L2 (set L2 (key t1) (add (value t1) (get L2 (key t1)))))
                     (new-L1 (rest-terms L1)))
                 (add-terms new-L1 new-L2))
               (add-terms (rest-terms L1) (set L2 (key t1) (value t1))))))))


add-terms

In [5]:
(define (mul-terms L1 L2)
  (if (empty-termlist? L1)
      (the-empty-termlist)
      (add-terms (mul-term-by-all-terms (first-term L1) L2)
                 (mul-terms (rest-terms L1) L2))))

(define (mul-term-by-all-terms t1 L)
  (if (empty-termlist? L)
      (the-empty-termlist)
      (let ((t2 (first-term L)))
        (let ((new-L (set (rest-terms L) (combine (key t1) (key t2)) (mul (value t1) (value t2)))))
          (mul-term-by-all-terms t1 new-L)))))


mul-term-by-all-terms

Terms can be any data struture that is `Map` and `keys` are symbols like `'a3b4c5`.

We transform string 'a3b4c5 (which means $a^{3}b^{4}c^{6}$) to $3a + 4b + 6c$, so the product procedure $a^{3}b^{4}c^{6} * a^{2}c^{1}f^{7}$ would be sum,

$3a + 4b + 6c + 2a + 1c + 7f = 5a + 4b + 7c + 7f$, 

Then we transform sum back to product: $a^{5}b^{4}c^{7}f^{7}$.



In [8]:
(define (combine K1 K2)
  (terms->symbols (add-terms (symbols->terms K1) (symbols->terms K2))))

(define (symbols->terms S)
  (string->terms (symbol->string S)))

(define (string->terms str)
  (define (helper a map)
    (if (< a (- (string-length str) 1))
        (set map (string->symbol (string-ref str a)) (string-ref str (+ a 1)))
        map))
  (helper 0 the-empty-termlist))
  

string->terms

We will not bother implement `Map`. Using list to represent `map` is not good either.

One last thing, we should provide a way to raise the single variable polynomial to mult-var polynomial.

We will not implement `get-all-var-set`, `sum-term`, `raise-order`. 

`get-all-var-set` is to find all the indeterminates both in coeff and variable and return as a set.

`sum-term` is sum of the `add-terms` procedure.

`raise-order` is to transform the variable list from orignal polynomial to mult-var-polynomial. Like below:

$x^{5}+2x^{4}+3x^{2}-2x^{1}-5$ 

((5 1)(4 2)(2 3)(1 -2)(0 -5)) -> ((((x 5)) 1)(((x 4)) 2)(((x 2)) 3)(((x 1)) -2)(((x 0)) -5))

$a^{5}b^{4}c^{7}f^{7}$. ((((a 5)(b 4)(c 7)(f 7)) 1))

In [2]:
(define (raise poly)
  (make-poly-mult-var (get-all-var-set poly) (sum-term (map (lambda (x) (mul-terms (raise (coeff x)) (raise-order (order x)))) term-list))))


raise

Other ref: https://github.com/kana/sicp/blob/master/ex-2.92.scm

http://jots-jottings.blogspot.com/2012/06/sicp-exercise-292-dealing-with_24.html